In [257]:
import pandas as pd
import numpy as np

## Сначала создали таблицу с данными из тестового задания в нормально читаемом формате csv и подгружаем ее для дальнейшей работы

In [258]:
df = pd.read_csv('Data_clients.csv')
df = df.drop(columns='client_id')
df.head(5)

,first_name,last_name,gender,age,city,rented_equipment,rental_period,rental_start_date,rental_end_date,late_return,registration,debt_history
0,Иван,Петров,M,35,Moscow,Drill,5 days,01/01/2023,01/05/2023,0,2,0
1,Мария,Сидорова,F,28,SPb,Ladder,3 days,03/01/2023,01/06/2023,1,1,3
2,Алексей,Орлов,M,42,Kazan,Saw,7 days,02/02/2023,02/09/2023,0,0,1
3,Ольга,Смирнова,F,25,Moscow,Sander,2 days,05/02/2023,02/12/2023,1,2,4
4,Сергей,Егоров,M,37,Rostov,Drill,4 days,09/02/2023,02/13/2023,0,1,2


## Функция для преобразования строки в количество дней

In [259]:
import re
# Функция для преобразования строки в количество дней
def extract_days(rental_period):
    match = re.search(r'\d+', rental_period)
    if match:
        return int(match.group())
    else:
        return None

## Приводим в целочисленный вид столбец с количеством дней аренды

In [260]:
df['rental_period'] = df['rental_period'].apply(extract_days)
df.head(5)

,first_name,last_name,gender,age,city,rented_equipment,rental_period,rental_start_date,rental_end_date,late_return,registration,debt_history
0,Иван,Петров,M,35,Moscow,Drill,5,01/01/2023,01/05/2023,0,2,0
1,Мария,Сидорова,F,28,SPb,Ladder,3,03/01/2023,01/06/2023,1,1,3
2,Алексей,Орлов,M,42,Kazan,Saw,7,02/02/2023,02/09/2023,0,0,1
3,Ольга,Смирнова,F,25,Moscow,Sander,2,05/02/2023,02/12/2023,1,2,4
4,Сергей,Егоров,M,37,Rostov,Drill,4,09/02/2023,02/13/2023,0,1,2


In [261]:
df['city'].value_counts()

city
Moscow             7
Rostov             4
SPb                3
Yekaterinburg      3
Novosibirsk        3
Kazan              2
Perm               2
Nizhny Novgorod    1
Krasnoyarsk        1
Ufa                1
Voronezh           1
Omsk               1
Samara             1
St. Petersburg     1
Name: count, dtype: int64

## При анализе столбца city увидел, что есть строка St. Petersburg и SPb, а это один и тот же город. Просизвожу замену

In [262]:
# Заменяем 'St. Petersburg' на 'SPb'
df['city'] = df['city'].replace('St. Petersburg', 'SPb')
df.head(5)

,first_name,last_name,gender,age,city,rented_equipment,rental_period,rental_start_date,rental_end_date,late_return,registration,debt_history
0,Иван,Петров,M,35,Moscow,Drill,5,01/01/2023,01/05/2023,0,2,0
1,Мария,Сидорова,F,28,SPb,Ladder,3,03/01/2023,01/06/2023,1,1,3
2,Алексей,Орлов,M,42,Kazan,Saw,7,02/02/2023,02/09/2023,0,0,1
3,Ольга,Смирнова,F,25,Moscow,Sander,2,05/02/2023,02/12/2023,1,2,4
4,Сергей,Егоров,M,37,Rostov,Drill,4,09/02/2023,02/13/2023,0,1,2


## Преобразуем календарные данные и создаем новый столбец по длительности аренды

In [263]:
# Преобразуем столбцы 'rental_start_date' и 'rental_end_date' в объекты datetime
df['rental_start_date'] = pd.to_datetime(df['rental_start_date'], dayfirst=True, errors='coerce')
df['rental_end_date'] = pd.to_datetime(df['rental_end_date'], errors='coerce')

# Создаем новый признак 'rental_duration' - продолжительность аренды
df['rental_duration'] = (df['rental_end_date'] - df['rental_start_date']).dt.days

df.head(5)

,first_name,last_name,gender,age,city,rented_equipment,rental_period,rental_start_date,rental_end_date,late_return,registration,debt_history,rental_duration
0,Иван,Петров,M,35,Moscow,Drill,5,2023-01-01,2023-01-05,0,2,0,4
1,Мария,Сидорова,F,28,SPb,Ladder,3,2023-01-03,2023-01-06,1,1,3,3
2,Алексей,Орлов,M,42,Kazan,Saw,7,2023-02-02,2023-02-09,0,0,1,7
3,Ольга,Смирнова,F,25,Moscow,Sander,2,2023-02-05,2023-02-12,1,2,4,7
4,Сергей,Егоров,M,37,Rostov,Drill,4,2023-02-09,2023-02-13,0,1,2,4


## Удаляем ненужные столбцы и кодируем пол

In [264]:
# Удаляем столбцы 'rental_start_date', 'rental_end_date','first_name', 'last_name'
df = df.drop(columns={'rental_start_date','rental_end_date','first_name', 'last_name'})

# Заменяем мужской и женский пол на цифры 1 и 0.
df['gender'] = df['gender'].map({'M': 0, 'F': 1})

df.head(5)

,gender,age,city,rented_equipment,rental_period,late_return,registration,debt_history,rental_duration
0,0,35,Moscow,Drill,5,0,2,0,4
1,1,28,SPb,Ladder,3,1,1,3,3
2,0,42,Kazan,Saw,7,0,0,1,7
3,1,25,Moscow,Sander,2,1,2,4,7
4,0,37,Rostov,Drill,4,0,1,2,4


## Кодируем текстовые столбцы

In [265]:
df = pd.get_dummies(df, columns=['city', 'rented_equipment'], prefix=['city', 'equipment'])

df.head(5)

,gender,age,rental_period,late_return,registration,debt_history,rental_duration,city_Kazan,city_Krasnoyarsk,city_Moscow,...,equipment_Jack Hammer,equipment_Ladder,equipment_Nail Gun,equipment_Paint Sprayer,equipment_Perforator,equipment_Sander,equipment_Saw,equipment_Spray Gun,equipment_Tile Cutter,equipment_Wall Sander
0,0,35,5,0,2,0,4,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1,1,28,3,1,1,3,3,False,False,False,...,False,True,False,False,False,False,False,False,False,False
2,0,42,7,0,0,1,7,True,False,False,...,False,False,False,False,False,False,True,False,False,False
3,1,25,2,1,2,4,7,False,False,True,...,False,False,False,False,False,True,False,False,False,False
4,0,37,4,0,1,2,4,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [266]:
X = df.drop(columns='late_return')
y = df['late_return']

## Поделим данные на тренировочную и тестовую выборки, чтобы можно было оценить точность модели

In [267]:
from sklearn.model_selection import train_test_split

In [268]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [269]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

# Создание и обучение модели Logistic Regression
model = LogisticRegression()
model.fit(X_train, y_train)

# Предсказания на валидационном наборе
pred = model.predict(X_test)

In [270]:
f1_score(y_test, pred)

0.8571428571428571

In [271]:
from sklearn.metrics import confusion_matrix

print(classification_report(y_test, pred, digits=4))
tn, fp, fn, tp = confusion_matrix(pred, y_test).ravel()
print(f'true negative: {tn}')
print(f'false positive: {fp}')
print(f'false negative: {fn}')
print(f'true positive: {tp}')

              precision    recall  f1-score   support

           0     1.0000    0.8750    0.9333         8
           1     0.7500    1.0000    0.8571         3

    accuracy                         0.9091        11
   macro avg     0.8750    0.9375    0.8952        11
weighted avg     0.9318    0.9091    0.9126        11

true negative: 7
false positive: 0
false negative: 1
true positive: 3


## Итог: точность предсказания модели 90%. Правильно определено 10 из 11. Только одна ошибка. Возможно при большем количестве данных можно было бы достичь больших результатов